# Cleaning Minerals Pull Data for Minerals Field Inspections

Cat Schooley  
GIS Analyst  
October 25, 2021

The Division of Oil, Gas, and Mining uses the ESRI product Survey123 in our field inspections. This application creates a streamlined workflow for conducting complicated inspections in the field and connecting them with databases back in the office and on the AGOL cloud server. 

One of the capabilities of Survey123 is a feature called 'Pull Data'. I should note this is only available in Survey123 Connect, not the online survey creator. Pull data allows the creators of the survey to connect a seperate csv file to the backend of the survey that the survey can then pull information from to make the inspector's jobs easier. For the minerals inspections the pull data table contains all the information for a mine (permit number, operator, etc.), but the inspector only has to input the permit number in the actual survey. Thanks to the pull data table all other mine information is autofilled and the inspector can get right to work on the inspection itself. It may seem miniscle, but this approach leads to less errors caused by typos, provides up-to-date information that the inspector might not know. Also a minute saved at the beginning of each survey adds up. 

I'll break down the code and why it's written the way it is. 

### Pulling Data into the Python Script <br>

For the minerals inspections the pull data comes from the FoxPro database (may have changed since time this was written). When a new mine is permitted, one is retired, bond amounts change or anything else is changed about a mine, this database is where those changes are stored. One of our inspectors (Kim Courbon at the time of this writing), takes data from the FoxPro database and saves it into a Google Sheet. This google sheet is easily shared but an issue occurs when she makes any changes. Due to the old software used in FoxPro, Kim has to completely overwrite the google sheet which changes the sheet's id. See the highlighted section below. 

    docs .google.com /spreadsheets/d/{SPREADSHEETID}/export? 
    format=csv&id{SPREADSHEETID}&`**gid=2113599541**` < Notice this section here

Though it is possible to pull down a google sheet as a csv to use in a python script, if this google sheet is overwritten often, you'll have to go a different route. 

As always, pull in the module you need for the script. For the first part of this script we use the pandas module. Pandas is often pulled in as pd. I'm not sure why, but it's what we do. 

In [ ]:
### import modules ###

import pandas as pd

#### Pull Data sheet directly from Google Sheet using the url <br>

First, I'll show you how to pull directly from the google sheet. As stated above, this doesn't work well for this example but can be used for other workflows. To do this you have to write the url the same as the url when exporting the google sheet to a csv. Google has been known to change this formatting though so if it doesn't work check out <a href="https://stackoverflow.com/"> Stack Overflow</a> or other online resources for an updated version. 

When you visit the google sheet you'll see a url that looks like this:

```python
https://docs.google.com/spreadsheets/d/{SPREADSHEETID}/edit#gid=2113599541
```

You will have to insert this between the spreadsheet id and the page id: 
```python
/export?format=csv&id{SPREADSHEETID}&
```

In order to get this final url to use in the code:

```python
https://docs.google.com/spreadsheets/d/{SPREADSHEETID}/export?format=csv&id{SPREADSHEETID}&gid=2113599541
```

Now that we have the url we need we can go ahead and code it in.

```python
url = "https://docs.google.com/spreadsheets/d/{SPREADSHEETIDENTIFIER}/export?format=csv&id{SPREADSHEETIDENTIFIER}&gid=2113599541"
```

I'm selecting for the columns that I want and not filtering out any NA values
```python
df = pd.read_csv(url, dtype = object, usecols= [3, 4, 5, 6, 7, 8, 9, 10, 11, 12], na_filter = False)
```

#### Pull Data from local Google Drive Desktop location <br>

Now I'm going to show you how to do it using <a href= "https://www.google.com/drive/download/">Google Drive Desktop</a> which circumvents the issue of a changing page id. From this point forward, you can run all the cells to run the script as we go along, simply change the file name and information to your computer's location. 

In [1]:
# This is my T:/ Drive, where the excel workbook is saved

file = 'T:/My Drive/FoxProLinked4.xlsm'

# Here I call the file, the sheet from the workbook, the dtype for the dataframe and the columns 

df = pd.read_excel(file, sheet_name="For Pull Data", dtype = str, usecols= [3, 4, 5, 6, 7, 8, 9, 10, 11, 12], na_filter = False, index_col= None)

# This is a bit redundant but makes sure that the data in the dataframe is read as a 'string'
df = df.astype(str)

NameError: name 'pd' is not defined

#### Remove Whitespaces and commas <br>

When data is pulled into pandas you should do a cleaning of the data. Python is sensitive to white spaces and tables can be sensitive to some special characters such as commas in the case of csvs. Keep that in mind whenever performing tasks like this. 

`str.strip` is used in the function below. This is why it was so important to have the data in the dataframe read as a string. Now you'll see if you were to use `df.dtypes` you would see that each field is of dtype object. This is because there's some columns in FoxPro that have both numbers and letters as well as some columns with large empty spaces. There is also a total row that appears at the bottom. All of these make it difficult for pandas to know if these of of type `str`, `int`, or something else so it chooses the catch-all `object`. Since we have the dataframe reading as a string, we can use the `.str` function and `.replace` function to clean up our data.

In [ ]:
def whitespaceRemover(dataframe):
    
    # iterating over the columns
    for i in dataframe.columns:
          
        # checking datatype of each columns
        if dataframe[i].dtype == 'object': #helps pinpoint if something is wrong with the data type
              
            # applying strip function on column
            dataframe[i] = dataframe[i].map(str.strip)
            
            dataframe[i] = dataframe[i].str.replace(",","")
            
        else:
              
            # if condn. is False then it will do nothing.
            pass
            
            print(f"{i} was skipped because field value was not a string")
        
### End Function ###

print("Commas and whitespaces being removed...")

whitespaceRemover(df) # plug your dataframe into function

### Overwriting Pull Data <br>

After it's clean we can write it to a csv file overwriting the previous pull data table in your survey. Notice the location of I am writing the new csv to. The Media folder holds items used in various ArcGIS Survey123 workflows. The Media folder can contain offline basemaps that survey authors want downloaded with a specific survey in the ArcGIS Survey123 Field App. Additional workflows that use the Media folder include image questions that use the draw and annotate appearance, or consuming images directly in your survey. 

Please note that all care should be taken to ensure the files you are updating with this script have the exact name as the files in the originally published media folder. Also, the files should have the same format, with the same field names as the original files (in the case of CSV files). Only rows of data should be updated or additional rows added. If you want to change the format of the files (rename or add columns) you should update the files via Connect in the media folder, update the survey XLSForm, and then re-publish the survey to ensure the changes do not break anything.

It is recommended to test the script on a backup copy of your survey and ensure the survey can be downloaded and updated in the field app, checking that the external choice lists and/or media items are working as expected. This should be confirmed before running the script and updating the media folder and files on your real survey that is currently in use by other ArcGIS Survey123 Field App users.

In [ ]:
print("Overwriting old Minerals Pulldata with cleaned Minerals Pulldata...")

df.to_csv(r"C:\Users\cschooley\ArcGIS\My Survey Designs\PullUpdate\MineralsPulldata.csv", index = False) 

#### Replacing Survey123 Pull Data with Local Update

To start we will import the required modules and define our variables.<br>
**Please format folder directories as C:/Users/username… include a trailing slash after the final folder name as the updated file directory is concatenated to the updated file name**


The variables are defined as follows: 

* **portalURL** - The URL for your WebGIS Portal (ex. www.arcgis.com)
* **username** - Your WebGIS Portal username (ex. gisadmin)
* **password** - You WebGIS Portal password (ex. gisadmin1)
* **itemID** - The Item ID for the ArcGIS Survey123 Form Item in you WebGIS Portal (ex. 89bc8c7844e548e09baa3aad4695e78b)
* **download_folder** - The folder directory you would like the Survey123 Form Item to be downloaded with the trailing slash (ex. C:/temp/)
* **updated_file** - The updated file name containing the extension (ex. myphoto.png)
* **source_loc** - Folder directory where the updated file is located (ex. C:/users/username/arcgis/my survey designs/)

In [ ]:
### Modules ###

import arcgis
from arcgis.gis import GIS
import zipfile
import shutil
import arcpy
import os

token = arcpy.GetSigninToken()
if token is not None:
    print(token['token'])

portalURL = r'https://arcgis.com'
username = 'OGMGIS_utahDNR'
password = 'MiningData801'
itemID = 'd1d980978a0e440fb371afbee891f80c'  #'5a5dbe18e9554e8185b87b137d66318a' Form 1     #'d1d980978a0e440fb371afbee891f80c' Real Mineral Survey ID
download_folder = r'C:/temp/'
updated_file = r'MineralsPulldata.csv'
source_loc = r'C:\Users\cschooley\ArcGIS\My Survey Designs\PullUpdate/'


### Connect to GIS ###

gis = GIS(portalURL, username, password, verify_cert=False)

#### Download the survey

We will start by grabbing the properties of the Survey123 Form Item. These properties are used later when we update the Form Item with a zip file containing the new media content.

In [ ]:
survey_manager = arcgis.apps.survey123.SurveyManager(gis)
surveyId = survey_manager.get(itemID)
surveyProp = surveyId.properties

# Find the Form item in the gis and download as a zip file to the *download_folder* directory.

itm = arcgis.gis.Item(gis,itemID)
print(itm)
savedZip = itm.download(save_path=download_folder)

Extract the zip file to an *`_extracted`* folder in the download location.
This *`_extracted`* folder is where the updated media files will be copied and rezipped later on. 

In [ ]:
def extractZIP(filename,folder):
    zfile = zipfile.ZipFile(filename)
    zfile.extractall(folder)

extractZIP(savedZip, download_folder + "_extracted/")

Copy the updated file to the media folder replacing the old file. 

In [ ]:
source_file = source_loc + updated_file
dest_file = download_folder + "_extracted/esriinfo/media/" + updated_file
shutil.copyfile(source_file, dest_file)
print (updated_file + " updated to: " + download_folder + "_extracted/esriinfo/media/")

Delete the old zip file that was previously downloaded. This will prevent any namespace issues and ensure the process of zipping and uploading the updated survey goes smoothly. 

In [ ]:
os.remove(savedZip)
print ("Old zip file deleted from: " + download_folder)

#### Upload the updated survey<br>

We will now zip the updated survey and place it in the download folder. The code below grabs the survey title from the survey properties and passes it into the function which zips the updated survey contents to the download folder.

In [ ]:
zipFileName = surveyProp['title']
os.chdir(download_folder)
updateZip = shutil.make_archive(zipFileName, 'zip', download_folder + '_extracted/')
print (updateZip)

Upload the new zip file and update the Form Item with the new Media folder content.

Then, clean up intermediate data. This process will delete the updated zip file as well as the extracted folder containing the unzipped survey content.

In [ ]:
itm.update({},updateZip)

os.remove(updateZip)
print (zipFileName + " deleted from: " + download_folder)

shutil.rmtree(download_folder + "_extracted/")
print ("extracted folder deleted from: " + download_folder)
print (zipFileName + " successfully updated with " + source_file + " and uploaded to your ArcGIS portal!")

### Send Email Notifications

In [ ]:
### Modules ###

import yagmail
from datetime import date
from datetime import datetime

print("Sending Emails...")
now = datetime.now()
date = now.strftime("%m/%d/%Y")
time = now.strftime("%I:%M:%S %p")
reciever = ['cschooley@utah.gov', 'michaelvanhatten@utah.gov', 'kcoburn@utah.gov']
body = f'Hello,\n\nThe Mineral Pulldata for {zipFileName} survey successfully updated with the latest Pulldata available as of {date} and uploaded to OGMGIS_utahDNR admin account!\n\nCompleted on {date} at {time}.\n\nThank you!'
yag = yagmail.SMTP("utahogmgisupdates@gmail.com", 'OGMG!Sutah801')
for recipient in reciever:
    yag.send(
        to=recipient,
        subject='Minerals Pulldata Update',
        contents = body
    )

print('Emails sent successfully.')

# Full Code

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


#################################### Remove Commas and Trailing/Leading Spaces from CSV ##############################

### import modules ###

import pandas as pd
import os

### Read CSV from Google Sheets #

### Pull Data sheet directly from Kim's FoxProLinked4

# url = "https://docs.google.com/spreadsheets/d/1abAxgx3VfBJLwPhPtlItr3gR85IUYuAQ/export?format=csv&id=1abAxgx3VfBJLwPhPtlItr3gR85IUYuAQ&gid=2113599541"

# select for the columns needed for Pulldata csv
# df = pd.read_csv(url, dtype = object, usecols= [3, 4, 5, 6, 7, 8, 9, 10, 11, 12], na_filter = False)

# Read from local excel workbook synced with Google Drive

file = 'T:/My Drive/FoxProLinked4.xlsm'

df = pd.read_excel(file, sheet_name="For Pull Data", dtype = str, usecols= [3, 4, 5, 6, 7, 8, 9, 10, 11, 12], na_filter = False, index_col= None)

df = df.astype(str)

### Function to remove whitespaces (removes leading and trailing spaces) ###

# Creating a function which will remove extra leading 
# and tailing whitespace from the data.
# pass dataframe as a parameter here

def whitespaceRemover(dataframe):
    
    # iterating over the columns
    for i in dataframe.columns:
          
        # checking datatype of each columns
        if dataframe[i].dtype == 'object':
              
            # applying strip function on column
            dataframe[i] = dataframe[i].map(str.strip)
            
            dataframe[i] = dataframe[i].str.replace(",","")
            
        else:
              
            # if condn. is False then it will do nothing.
            pass
            
            print(f"{i} was skipped because field value was not a string")
        
### End Function ###

print("Commas and whitespaces being removed...")

whitespaceRemover(df) # plug your dataframe into function

### Write to new csv ###
                  
print("Overwriting old Minerals Pulldata with cleaned Minerals Pulldata...")

### Due to the requirement to zip and unzip files, this has to be on someone's PC instead of on a cloud server
### This could be worked around if there was an API we could use for zipping and unzipping maybe... Beyond my
### knowledge but worth looking into further if we would like

df.to_csv(r"C:\Users\cschooley\ArcGIS\My Survey Designs\PullUpdate\MineralsPulldata.csv", index = False) # Use full path, same name as previous pulldata used in survey

################################## Update Pulldata for Minerals ############################

# The Media folder holds items used in various ArcGIS Survey123 workflows.
# The Media folder can contain offline basemaps that survey authors want downloaded with a specific survey in the ArcGIS Survey123 Field App.
# Additional workflows that use the Media folder include image questions that use the draw and annotate appearance,
# or consuming images directly in your survey. 
# As data and workflows advance, it's common that folks want to update the information in the media folder,
# but do not want to re-publish their survey every time there are changes to a file.
# This sample notebook automates the task of updating the contents in the media folder associated with a published survey. 
# 
# **IMPORTANT**: Please note that all care should be taken to ensure the files you are updating with this script have
# the exact name as the files in the originally published media folder.
# Also, the files should have the same format, with the same field names as the original files (in the case of CSV files).
# Only rows of data should be updated or additional rows added.
# If you want to change the format of the files (rename or add columns) you should update the files
# via Connect in the media folder, update the survey XLSForm,
# and then re-publish the survey to ensure the changes do not break anything.
# 
# It is recommended to test the script on a backup copy of your survey and ensure the survey
# can be downloaded and updated in the field app, checking that the external choice lists and/or media items are working
# as expected. This should be confirmed before running the script and updating the media folder and files on your real
# survey that is currently in use by other ArcGIS Survey123 Field App users.
# 
# **Note**: To use this notebook as a python script, check **Insert link HERE**.
# Running this script from Windows Task Scheduler or ArcGIS Pro scheduled tools are effective for workflows
# that require regular updates. 
# 
# To start we will import the required modules and define our variables.
# **Please format folder directories as C:/Users/username… include a trailing slash after the final folder
# name as the updated file directory is concatenated to the updated file name**.
# 
# The variables are defined as follows: 
# 
# * **portalURL** - The URL for your WebGIS Portal (ex. www.arcgis.com)
# * **username** - Your WebGIS Portal username (ex. gisadmin)
# * **password** - You WebGIS Portal password (ex. gisadmin1)
# * **itemID** - The Item ID for the ArcGIS Survey123 Form Item in you WebGIS Portal (ex. 89bc8c7844e548e09baa3aad4695e78b)
# * **download_folder** - The folder directory you would like the Survey123 Form Item to be downloaded with the trailing slash (ex. C:/temp/)
# * **updated_file** - The updated file name containing the extension (ex. myphoto.png)
# * **source_loc** - Folder directory where the updated file is located (ex. C:/users/username/arcgis/my survey designs/)

### import modules ###

import arcgis
from arcgis.gis import GIS
import zipfile
import shutil
import arcpy

token = arcpy.GetSigninToken()
if token is not None:
    print(token['token'])

portalURL = r'https://arcgis.com'
username = 'OGMGIS_utahDNR'
password = 'MiningData801'
itemID = 'd1d980978a0e440fb371afbee891f80c'  #'5a5dbe18e9554e8185b87b137d66318a' Form 1     #'d1d980978a0e440fb371afbee891f80c' Real Mineral Survey ID
download_folder = r'C:/temp/'
updated_file = r'MineralsPulldata.csv'
source_loc = r'C:\Users\cschooley\ArcGIS\My Survey Designs\PullUpdate/'


### Connect to GIS ###

gis = GIS(portalURL, username, password, verify_cert=False)


# ## Download the survey
# 
# We will start by grabbing the properties of the Survey123 Form Item.
# These properties are used later when we update the Form Item with a zip file containing the new media content.

survey_manager = arcgis.apps.survey123.SurveyManager(gis)
surveyId = survey_manager.get(itemID)
surveyProp = surveyId.properties

# Find the Form item in the gis and download as a zip file to the *download_folder* directory.

itm = arcgis.gis.Item(gis,itemID)
print(itm)
savedZip = itm.download(save_path=download_folder)


# Extract the zip file to an *`_extracted`* folder in the download location.
# This *`_extracted`* folder is where the updated media files will be copied and rezipped later on. 


def extractZIP(filename,folder):
    zfile = zipfile.ZipFile(filename)
    zfile.extractall(folder)

extractZIP(savedZip, download_folder + "_extracted/")


# Copy the updated file to the media folder replacing the old file. 

source_file = source_loc + updated_file
dest_file = download_folder + "_extracted/esriinfo/media/" + updated_file
shutil.copyfile(source_file, dest_file)
print (updated_file + " updated to: " + download_folder + "_extracted/esriinfo/media/")


# Delete the old zip file that was previously downloaded. This will prevent any namespace issues and ensure the
# process of zipping and uploading the updated survey goes smoothly. 

os.remove(savedZip)
print ("Old zip file deleted from: " + download_folder)


# ## Upload the updated survey
# 
# We will now zip the updated survey and place it in the download folder. The code below grabs the survey title from
# the survey properties and passes it into the function which zips the updated survey contents to the download folder.

zipFileName = surveyProp['title']
os.chdir(download_folder)
updateZip = shutil.make_archive(zipFileName, 'zip', download_folder + '_extracted/')
print (updateZip)


# Upload the new zip file and update the Form Item with the new Media folder content.

itm.update({},updateZip)

# Clean up intermediate data. This process will delete the updated zip file as well as the extracted folder
# containing the unzipped survey content.

os.remove(updateZip)
print (zipFileName + " deleted from: " + download_folder)

shutil.rmtree(download_folder + "_extracted/")
print ("extracted folder deleted from: " + download_folder)
print (zipFileName + " successfully updated with " + source_file + " and uploaded to your ArcGIS portal!")

################### Send Email Notifications ####################

import yagmail
from datetime import date
from datetime import datetime

print("Sending Emails...")
now = datetime.now()
date = now.strftime("%m/%d/%Y")
time = now.strftime("%I:%M:%S %p")
reciever = ['cschooley@utah.gov', 'michaelvanhatten@utah.gov', 'kcoburn@utah.gov']
body = f'Hello,\n\nThe Mineral Pulldata for {zipFileName} survey successfully updated with the latest Pulldata available as of {date} and uploaded to OGMGIS_utahDNR admin account!\n\nCompleted on {date} at {time}.\n\nThank you!'
yag = yagmail.SMTP("utahogmgisupdates@gmail.com", 'OGMG!Sutah801')
for recipient in reciever:
    yag.send(
        to=recipient,
        subject='Minerals Pulldata Update',
        contents = body
    )

print('Emails sent successfully.')

# %%
